<a href="https://colab.research.google.com/github/eyaler/avatars4all/blob/master/mraa-body.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo for paper "Motion Representations for Articulated Animation"

## **Full body Tai chi and TED models**

### Made just a little bit more accessible by Eyal Gruss (https://eyalgruss.com, eyalgruss@gmail.com)

##### Original project: https://snap-research.github.io/articulated-animation

##### Original notebook: https://colab.research.google.com/github/AliaksandrSiarohin/articulated-animation/blob/main/demo.ipynb

##### My avatars4all repository: https://github.com/eyaler/avatars4all
##### A list of more generative tools: https://j.mp/generativetools

In [ ]:
#@title Setup
 
%cd /content
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install
!git clone https://github.com/eyaler/articulated-animation/

from demo import load_checkpoints

generator, region_predictor, avd_network = load_checkpoints(config_path='config/ted384.yaml',
                                                            checkpoint_path='checkpoints/ted384.pth')

!git clone --depth 1 https://github.com/eyaler/first-order-model
import os
if not os.path.exists('/content/taichi-adv-cpk.pth.tar'):
  !gdown https://drive.google.com/uc?id=1K_XBKgQhVY5febYaNjT18GBp9FLWMRbz -O /content/taichi-adv-cpk.pth.tar
!wget --no-check-certificate -nc https://eyalgruss.com/fomm/taichi-adv-cpk.pth.tar -O /content/taichi-adv-cpk.pth.tar
if not os.path.exists('/content/fashion.pth.tar'):
  !gdown https://drive.google.com/uc?id=1RSekVOZxwAe_G48iEUd4pjg1kzHjnnUC -O /content/fashion.pth.tar
!wget --no-check-certificate -nc https://eyalgruss.com/fomm/fashion.pth.tar -O /content/fashion.pth.tar
!pip install -U youtube-dl
!pip install -U imageio
!pip install -U imageio-ffmpeg

In [ ]:
#@title Choose model
model = 'taichi256' #@param ['taichi256', 'ted384', 'ted-youtube384']

res = int(model[-3:])
from demo import load_checkpoints
generator, region_predictor, avd_network = load_checkpoints(config_path='config/%s.yaml'%model,
                                                            checkpoint_path='checkpoints/%s.pth'%model)

In [ ]:
#@title Get the Driver video and Avatar image from the web
#@markdown 1. You can change the URLs to your **own** stuff!
#@markdown 2. Alternatively, you can upload **local** files in the next cells
 
video_url = 'https://www.youtube.com/watch?v=NdEPmitJvaA' #@param {type:"string"}
image_url = 'https://i.pinimg.com/564x/74/9e/96/749e96293399ebe1b8c99d89be522383.jpg' #@param {type:"string"}
 
if video_url:
  !rm -f /content/video.mp4
  !youtube-dl -f 'bestvideo[ext=mp4][vcodec!*=av01][height<=360]+bestaudio[ext=m4a]/mp4[height<=360][vcodec!*=av01]/mp4[vcodec!*=av01]/mp4' '$video_url' --merge-output-format mp4 -o /content/video
  !mv /content/video.mp4 /content/video 
 
if image_url:
  !wget '$image_url' -O /content/image

In [ ]:
#@title Optionally upload local Driver video { run: "auto" }
manually_upload_video = False #@param {type:"boolean"}
if manually_upload_video:
  from google.colab import files
  import shutil

  %cd /content/sample_data
  try:
    uploaded = files.upload()
  except Exception as e:
    %cd /content
    raise e

  for fn in uploaded:
    shutil.move('/content/sample_data/'+fn, '/content/video')
    break
  %cd /content

In [ ]:
#@title Optionally upload local Avatar image { run: "auto" }
manually_upload_image = False #@param {type:"boolean"}
if manually_upload_image:
  from google.colab import files
  import shutil
 
  %cd /content/sample_data
  try:
    uploaded = files.upload()
  except Exception as e:
    %cd /content
    raise e

  for fn in uploaded:
    shutil.move('/content/sample_data/'+fn, '/content/image')
    break
  %cd /content

In [ ]:
#@title Optionally shorten Driver video
start_seconds = 0 #@param {type:"number"}
duration_seconds =  60#@param {type:"number"}
start_seconds = max(start_seconds,0)
duration_seconds = max(duration_seconds,0)
 
if duration_seconds:
  !mv /content/video /content/full_video
  !ffmpeg  -ss $start_seconds -t $duration_seconds -i /content/full_video -f mp4 /content/video -y

In [ ]:
#@title Prepare assets
#@markdown If you ran out of RAM this means that the video is too large. You can shorten it above.
 
center_video_to_body = True #@param {type:"boolean"}
crop_video_to_body = True #@param {type:"boolean"}
video_crop_expansion_factor = 1.05 #@param {type:"number"}
center_image_to_body = True #@param {type:"boolean"}
crop_image_to_body = False #@param {type:"boolean"}
image_crop_expansion_factor = 1.05 #@param {type:"number"}
video_crop_expansion_factor = max(video_crop_expansion_factor, 1)
image_crop_expansion_factor = max(image_crop_expansion_factor, 1)
 
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML, clear_output
import cv2
import warnings
warnings.filterwarnings("ignore")
 
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
 
def fix_dims(im):
    if im.ndim == 2:
        im = np.tile(im[..., None], [1, 1, 3])
    return im[...,:3]
 
def get_crop(im, center_body=True, crop_body=True, expansion_factor=1, rects=None):
    im = fix_dims(im)
    if (center_body or crop_body) and rects is None:
        rects, _ = hog.detectMultiScale(im, winStride=(4, 4),padding=(8,8), scale=expansion_factor)
    if (center_body or crop_body) and rects is not None and len(rects):
        x0,y0,w,h = sorted(rects, key=lambda x: x[2]*x[3])[-1]
        if crop_body:
            x0 += w//2-h//2
            x1 = x0+h
            y1 = y0+h
        else:
            img_h,img_w = im.shape[:2]
            x0 += (w-img_h)//2
            x1 = x0+img_h
            y0 = 0
            y1 = img_h
    else:
        h,w = im.shape[:2]
        x0 = (w-h)//2
        x1 = (w+h)//2
        y0 = 0
        y1 = h
    return int(x0),int(x1),int(y0),int(y1)
 
def pad_crop_resize(im, x0=None, x1=None, y0=None, y1=None, new_h=256, new_w=256):
    im = fix_dims(im)
    h,w = im.shape[:2]
    if x0 is None:
      x0 = 0
    if x1 is None:
      x1 = w
    if y0 is None:
      y0 = 0
    if y1 is None:
      y1 = h
    if x0<0 or x1>w or y0<0 or y1>h:
        im = np.pad(im, pad_width=[(max(-y0,0),max(y1-h,0)),(max(-x0,0),max(x1-w,0)),(0,0)], mode='edge')
    return resize(im[max(y0,0):y1-min(y0,0),max(x0,0):x1-min(x0,0)], (new_h, new_w))
 
 
source_image = imageio.imread('/content/image')
source_image = pad_crop_resize(source_image, *get_crop(source_image, center_body=center_image_to_body, crop_body=crop_image_to_body, expansion_factor=image_crop_expansion_factor), new_h=res, new_w=res)
 
with imageio.get_reader('/content/video', format='mp4') as reader:
  fps = reader.get_meta_data()['fps']
 
  driving_video = []
  rects = None
  try:
      for i,im in enumerate(reader):
          if not crop_video_to_body:
              break
          rects, _ = hog.detectMultiScale(im, winStride=(4, 4),padding=(8,8), scale=video_crop_expansion_factor)
          if rects is not None and len(rects):
              break
      x0,x1,y0,y1 = get_crop(im, center_body=center_video_to_body, crop_body=crop_video_to_body, expansion_factor=video_crop_expansion_factor, rects=rects)
      reader.set_image_index(0)
      for im in reader:
          driving_video.append(pad_crop_resize(im,x0,x1,y0,y1,new_h=res,new_w=res))
  except RuntimeError:
      pass
 
def vid_display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))
 
    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])
 
    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
 
clear_output()
if rects is not None and len(rects):
    print('first found body in frame %d'%i)
HTML(vid_display(source_image, driving_video).to_html5_video())

In [ ]:
#@title Animate
 
animation_mode = 'avd' #@param ['standard', 'relative', 'avd']

%cd /content/articulated-animation
import torch
from tqdm import tqdm
from skimage import img_as_ubyte
 
def make_animation(source_image, driving_video, generator, region_predictor, avd_network,
                   animation_mode='standard', cpu=False):
    with torch.no_grad():
        predictions = []
        source = torch.tensor(source_image[np.newaxis].astype(np.float32)).permute(0, 3, 1, 2)
        if not cpu:
            source = source.cuda()
        driving = torch.tensor(np.array(driving_video)[np.newaxis].astype(np.float32)).permute(0, 4, 1, 2, 3)
        source_region_params = region_predictor(source)
        driving_region_params_initial = region_predictor(driving[:, :, 0])

        for frame_idx in tqdm(range(driving.shape[2])):
            driving_frame = driving[:, :, frame_idx]
            if not cpu:
                driving_frame = driving_frame.cuda()
            driving_region_params = region_predictor(driving_frame)
            new_region_params = get_animation_region_params(source_region_params, driving_region_params,
                                                            driving_region_params_initial, avd_network=avd_network,
                                                            mode=animation_mode)
            out = generator(source, source_region_params=source_region_params, driving_region_params=new_region_params)

            predictions.append(np.transpose(out['prediction'].data.cpu().numpy(), [0, 2, 3, 1])[0])
    return predictions

predictions = make_animation(source_image, driving_video, generator, region_predictor, avd_network,
                              animation_mode=animation_mode) 
imageio.mimsave('/content/generated.mp4', [img_as_ubyte(frame) for frame in predictions], fps=fps)
!ffmpeg -i /content/generated.mp4 -i /content/video -c:v libx264 -c:a aac -map 0:v -map 1:a? -pix_fmt yuv420p -profile:v baseline -movflags +faststart /content/final.mp4 -y
 
clear_output()
HTML(vid_display(source_image, driving_video, predictions).to_html5_video())

In [ ]:
#@title Download

from google.colab import files
files.download('/content/final.mp4')